In [ ]:
%load_ext autoreload
%autoreload 2
# default_exp plugin.stateful

In [ ]:
#export
from pymemri.data.schema import Item, CVUStoredDefinition
from pymemri.plugin.pluginbase import PluginBase
import logging

In [ ]:
#hide
from nbdev.showdoc import *

# Stateful Plugins

If a plugin with states are going to be used, this includes the plugins with user interaction, then StatefulPlugin should be used.


In [ ]:
# export
RUN_IDLE = 'idle'                           #1
RUN_INITIALIZED = 'initilized'              #2
RUN_USER_ACTION_NEEDED = 'userActionNeeded' # 2-3
RUN_USER_ACTION_COMPLETED = 'ready'         # 2-3
RUN_STARTED = 'start'                       #3
RUN_FAILED = 'error'                        # 3-4
RUN_COMPLETED = 'done'                      #4

logging.basicConfig(format='%(asctime)s [%(levelname)s] - %(message)s')

In [ ]:
# export
class PersistentState(Item):
    """ Persistent state variables saved for plugin such as views, accounts, the last state to resume from etc. """

    properties = Item.properties + ["pluginId", "lastState"]
    edges = Item.edges + ["account", "view"]

    def __init__(self, pluginId=None, lastState=None, account=None, view=None, **kwargs):
        super().__init__(**kwargs)
        self.pluginId = pluginId
        self.lastState = lastState
        self.account = account if account is not None else []
        self.view = view if view is not None else []

In [ ]:
# export
class StatefulPlugin(PluginBase):
    """ Provides state/view setter and getter functions to plugin runtime """    
    
    properties = PluginBase.properties + ["runId"]
    edges = PluginBase.edges
    
    def __init__(self, runId, pluginState=None, **kwargs):
        super().__init__(**kwargs)
        self.runId = runId

    def get_persistent_state(self, client):
        run = self.get_run(client)
        states = run.get_edges("persistentState")
        if len(states) > 0:
            return states[0].traverse(run)

    def set_persistent_state(self, client):
        raise Exception("Not implemented")

    def initialized(self, client):
        logging.warning("PLUGIN run is initialized")
        self._set_run_vars(client, {'state':RUN_INITIALIZED})

    def started(self, client):
        logging.warning("PLUGIN run is started")
        self._set_run_vars(client, {'state':RUN_STARTED})

    def failed(self, client, error):
        logging.error(f"PLUGIN run is failed: {error}")
        print("Exception while running plugin:", error)
        self._set_run_vars(client, {'state':RUN_FAILED, 'message': str(error)})

    def completed(self, client):
        logging.warning("PLUGIN run is completed")
        self._set_run_vars(client, {'state':RUN_COMPLETED})

    def complete_action(self, client):
        self._set_run_vars(client, {'state': RUN_USER_ACTION_COMPLETED})

    def action_required(self, client):
        self._set_run_vars(client, {'state': RUN_USER_ACTION_NEEDED})

    def is_action_required(self, client):
        return self._get_run_state(client) == RUN_USER_ACTION_NEEDED

    def is_action_completed(self, client):
        return self._get_run_state(client) == RUN_USER_ACTION_COMPLETED

    def is_completed(self, client):
        return self._get_run_state(client) == RUN_COMPLETED

    def is_daemon(self, client):
        run = self.get_run(client, expanded=False)
        return run.interval and run.interval > 0

    def get_run(self, client, expanded=False):
        return client.get(self.runId, expanded=expanded)


    # =======================================
    # --------- INTERNAL METHODS ------------

    # EXPORT TO STATEFUL.py
    def _get_run_state(self, client):
        start_plugin = self.get_run(client)
        return start_plugin.state

    def _set_run_vars(self, client, vars):
        start_plugin = client.get(self.runId, expanded=False)
        for k,v in vars.items():
            setattr(start_plugin, k, v)
        client.update_item(start_plugin)
        
    def _get_run_view(self, client, run):
        run = self.get_run(client, expanded=True)
        if run:
            views = run.get_edges('view')
            if len(views) > 0:
                return views[0].traverse(run)
        
    def _set_run_view(self, client, view_name):
        run = self.get_run(client)
        views = client.search({'type': 'CVUStoredDefinition', 'name': view_name})
        if len(views) > 0:

            view = views[0]

            attached_CVU_edge = self._get_run_view(client, run) # index error here if there is no already bound CVU 
            if attached_CVU_edge:
                logging.warning(f"Plugin Run already has a view. Updating with {view_name}")
                attached_CVU_edge.target = view  # update CVU
                attached_CVU_edge.update(client) # having doubts if this really updates the existing edge
            else:
                logging.warning(f"Plugin Run does not have a view. Creating {view_name}")
                run.add_edge('view', view)
                run.update(client)
            return True
        return False

## Setting and communicating states through PluginRun

An example stateful plugin is below.

In [ ]:

class MyStatefulPlugin(StatefulPlugin):
    
    def __init__(self, runId=None, **kwargs):
        super().__init__(runId=runId, **kwargs)

    def run(self):
        print("Running plugin")
        pass
    
    def add_to_schema(self):
        print("Adding schema")
        pass

Through this inherited class you can set states to control the flow and views that are displayed to user.

In [ ]:
from pymemri.pod.client import PodClient
from pymemri.data.schema import CVUStoredDefinition
from pymemri.plugin.pluginbase import PluginRun, register_base_schemas

# prepare
client = PodClient()
register_base_schemas(client)

# create a run
run = PluginRun(containerImage="stateful_plugin", pluginModule="StatefulPlugin", pluginName="stateful_plugin")
client.create(run)

# test the run states    
stateful = MyStatefulPlugin(runId=run.id)
stateful.started(client)
assert stateful._get_run_state(client) == RUN_STARTED

# test run views
view = CVUStoredDefinition(name="test-view")
client.create(view)
assert stateful._set_run_view(client, "non-existent-view") == False
assert stateful._set_run_view(client, "test-view") == True
assert stateful._get_run_view(client, run).name == 'test-view'